In [189]:
import numpy as np
from tabulate import tabulate

In [190]:
np.random.seed(47)

In [191]:
actionsMap = { action: i for i, action in enumerate(['up', 'down', 'left', 'right']) }
reverseActionsMap = { v: k for k, v in actionsMap.items() }
actionsMap, reverseActionsMap

({'up': 0, 'down': 1, 'left': 2, 'right': 3},
 {0: 'up', 1: 'down', 2: 'left', 3: 'right'})

In [192]:
def getValidActions(pos):
    row, col = pos

    actions = []
    if row > 0:
        actions.append(actionsMap['up'])
    if row < 3:
        actions.append(actionsMap['down'])
    if col > 0:
        actions.append(actionsMap['left'])
    if col < 3:
        actions.append(actionsMap['right'])
    return actions

def move(currentState, action):
    row, col = currentState
    if action == actionsMap['up']:
        row -= 1
    elif action == actionsMap['down']:
        row += 1
    elif action == actionsMap['left']:
        col -= 1
    elif action == actionsMap['right']:
        col += 1
    return (row, col)

def getReward(curr):
    if curr == (1, 1):
        return -10
    elif curr == (3, 3):
        return 10
    else:
        return -1

In [193]:
qvalues = dict()
for i in range(4):
    for j in range(4):
        qvalues[(i, j)] = { a: 0 for a in getValidActions((i, j)) }
qvalues

{(0, 0): {1: 0, 3: 0},
 (0, 1): {1: 0, 2: 0, 3: 0},
 (0, 2): {1: 0, 2: 0, 3: 0},
 (0, 3): {1: 0, 2: 0},
 (1, 0): {0: 0, 1: 0, 3: 0},
 (1, 1): {0: 0, 1: 0, 2: 0, 3: 0},
 (1, 2): {0: 0, 1: 0, 2: 0, 3: 0},
 (1, 3): {0: 0, 1: 0, 2: 0},
 (2, 0): {0: 0, 1: 0, 3: 0},
 (2, 1): {0: 0, 1: 0, 2: 0, 3: 0},
 (2, 2): {0: 0, 1: 0, 2: 0, 3: 0},
 (2, 3): {0: 0, 1: 0, 2: 0},
 (3, 0): {0: 0, 3: 0},
 (3, 1): {0: 0, 2: 0, 3: 0},
 (3, 2): {0: 0, 2: 0, 3: 0},
 (3, 3): {0: 0, 2: 0}}

In [194]:
def generateEpisode():
    trajectory = []
    curr = (0, 0)
    while curr != (1, 1) and curr != (3, 3):
        # action = max(epsilon_soft_policy(qvalues[curr]), key= lambda x: x[1])[0]
        action = np.random.choice(getValidActions(curr))
        # print(f"at pos {curr} got action {reverseActionsMap[action]}")
        next = move(curr, action)
        trajectory.append(((curr, action), getReward(next)))
        curr = next

    return trajectory

for i in range(10):
    print(generateEpisode())

[(((0, 0), np.int64(3)), -1), (((0, 1), np.int64(3)), -1), (((0, 2), np.int64(1)), -1), (((1, 2), np.int64(0)), -1), (((0, 2), np.int64(1)), -1), (((1, 2), np.int64(3)), -1), (((1, 3), np.int64(0)), -1), (((0, 3), np.int64(2)), -1), (((0, 2), np.int64(2)), -1), (((0, 1), np.int64(2)), -1), (((0, 0), np.int64(3)), -1), (((0, 1), np.int64(3)), -1), (((0, 2), np.int64(3)), -1), (((0, 3), np.int64(1)), -1), (((1, 3), np.int64(1)), -1), (((2, 3), np.int64(0)), -1), (((1, 3), np.int64(0)), -1), (((0, 3), np.int64(2)), -1), (((0, 2), np.int64(3)), -1), (((0, 3), np.int64(2)), -1), (((0, 2), np.int64(2)), -1), (((0, 1), np.int64(2)), -1), (((0, 0), np.int64(1)), -1), (((1, 0), np.int64(1)), -1), (((2, 0), np.int64(0)), -1), (((1, 0), np.int64(3)), -10)]
[(((0, 0), np.int64(1)), -1), (((1, 0), np.int64(3)), -10)]
[(((0, 0), np.int64(1)), -1), (((1, 0), np.int64(1)), -1), (((2, 0), np.int64(0)), -1), (((1, 0), np.int64(0)), -1), (((0, 0), np.int64(3)), -1), (((0, 1), np.int64(2)), -1), (((0, 0),

In [195]:
# %%
def montecarlo(trajectories, qvalues, gamma=0.9):
    returns_sum = dict()
    returns_count = dict()

    for trajectory in trajectories:
        G = 0
        visited = set()

        # Work backwards to calculate returns
        for t in reversed(range(len(trajectory))):
            (state, action), reward = trajectory[t]
            G = gamma * G + reward

            if (state, action) not in visited:
                visited.add((state, action))
                
                if (state, action) not in returns_sum:
                    returns_sum[(state, action)] = 0
                    returns_count[(state, action)] = 0

                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1
                qvalues[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]


In [196]:
# %%
episodes = [generateEpisode() for _ in range(5000)]
montecarlo(episodes, qvalues, gamma=0.9)


In [197]:
qvalues

{(0, 0): {1: -9.205448440289553, 3: -9.333662502373135},
 (0, 1): {1: -10.0, 2: -9.400504633453211, 3: -8.367521730225363},
 (0, 2): {1: -7.532025612984717, 2: -9.36966442406668, 3: -7.182162298262367},
 (0, 3): {1: -5.414012293874806, 2: -7.941984101034038},
 (1, 0): {0: -9.288042160190543, 1: -8.030559424666993, 3: -10.0},
 (1, 1): {0: 0, 1: 0, 2: 0, 3: 0},
 (1, 2): {0: -7.9262737595681765,
  1: -4.436862767862954,
  2: -10.0,
  3: -5.391556338020939},
 (1, 3): {0: -6.965175577867602,
  1: -0.25905708293845403,
  2: -7.1084221702748875},
 (2, 0): {0: -9.34441155138185, 1: -6.670705604649064, 3: -7.35491553513547},
 (2, 1): {0: -10.0,
  1: -5.650931211610691,
  2: -8.403563532827595,
  3: -3.7788309408551926},
 (2, 2): {0: -6.929705129168563,
  1: 0.6757660523598975,
  2: -7.530833826236013,
  3: -0.004210412388726882},
 (2, 3): {0: -4.90051353660186, 1: 10.0, 2: -3.7546920884401285},
 (3, 0): {0: -7.843830714332813, 3: -4.677193859371143},
 (3, 1): {0: -6.731531918381403,
  2: -6.533

In [198]:
policy = []
for i in range(4):

    row = []
    for j in range(4):
        curr = (i, j)
        best_action = max(qvalues[curr].items(), key=lambda x: x[1])[0]
        best_action = reverseActionsMap[best_action]
        row.append(best_action)
    policy.append(row)
    row = []

policy[1][1] = 'x'
policy[3][3] = 'G'

print(tabulate(policy, tablefmt='grid'))


+-------+-------+-------+------+
| down  | right | right | down |
+-------+-------+-------+------+
| down  | x     | down  | down |
+-------+-------+-------+------+
| down  | right | down  | down |
+-------+-------+-------+------+
| right | right | right | G    |
+-------+-------+-------+------+
